In [23]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import matplotlib.pyplot as plt 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [238]:
## Loading DATA
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
SDATA = pd.read_csv("Shareholder97.csv",index_col=0)

# Conver date from string to jdatetime
SDATA['True_Date'] = pd.to_datetime(SDATA['True_Date'], format='%Y-%m-%d')
G = SDATA.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
DataOrg_date_GtoJ_dict = dict(zip(G,J))
SDATA['Jalali_Date']=SDATA.True_Date.map(DataOrg_date_GtoJ_dict)

SDATA.drop(columns=['Shareholder_raw', 'High', 'Low', 'Open', 'Last', 'Volume', 'close',
       'True_Date', 'year', 'month', 'day', 'Fill_Flag_HOLDER',
       'Fill_Flag_PRICE', 'Unadjusted_Close','chnk_id'],inplace=True)

In [269]:
# Creating Dataframe for saving concentration mearsurs
CMdf = SDATA.groupby('Symbol',as_index=False).agg({'Id_tse':'first','ShareHolder':'count'}).rename(columns={'ShareHolder':'Num_holders'})
CMdf.reset_index(drop=True,inplace=True)

# **Concentration Measures**

## 1- Largest Owner

In [270]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':'max'}).rename(columns={'percent':'Largest_Owner'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left')

## 2- First/Second

In [271]:
def nth_max(data,nth=1,interval=False):
    data = data.sort_values(ascending=False)
    if interval:
        return(np.round(data.iloc[min(nth[0]-1,len(data)-1):min(nth[1],len(data))],2))
    else:
        return(np.round(data.iloc[min(nth-1,len(data)-1)],2))

In [272]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/nth_max(x,nth=2,interval=False)}}).rename(columns={'percent':'First_Second'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Second', '<lambda>'):'First_Second'})

C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


## 3- First/Sumtwofour

In [273]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/sum(nth_max(x,nth=[2,4],interval=True))}}).rename(columns={'percent':'First_Sumtwofour'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Sumtwofour', '<lambda>'):'First_Sumtwofour'})

## 4- Sumfive

In [274]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum(nth_max(x,nth=[1,5],interval=True))}}).rename(columns={'percent':'Sumfive'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Sumfive', '<lambda>'):'Sumfive'})

In [275]:
CMdf

,Symbol,Id_tse,Num_holders,Largest_Owner,First_Second,First_Sumtwofour,Sumfive
0,اعتضاد غدیر,34973883374080119,1,99.89,1.000000,1.000000,99.89
1,کاغذ مراغه,21383339313241074,1,51.53,1.000000,1.000000,51.53
2,آ س پ,17617474823279712,5,40.26,1.963902,0.912925,89.20
3,آتیمس,22839330962768817,1,98.98,1.000000,1.000000,98.98
4,آرمان,38738476064699383,10,19.78,1.163529,0.444395,72.11
...,...,...,...,...,...,...,...
600,گنگین,59470107928175959,2,79.25,3.962500,3.962500,99.25
601,گوهران,65018804181564924,1,94.14,1.000000,1.000000,94.14
602,گپارس,59848307608894801,4,20.90,3.181126,2.279171,30.07
603,گکوثر,66599109405217136,4,46.00,1.380138,1.099952,87.82


In [277]:
SDATA[SDATA.Id_tse==38738476064699383]

,Symbol,Jalali_Date,ShareHolder,Id_tse,Industry,Industry_Id,percent,quantity,ShareHolder_Type,Shares,MarketCap
19776,آرمان,1397-12-28,اقتصادی نگین گردشگری ایرانیان,38738476064699383,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,66.0,4.06,48800000,private,1.500000e+09,1.116000e+12
19777,آرمان,1397-12-28,بانک تات,38738476064699383,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,66.0,17.00,204000000,bank,1.500000e+09,1.116000e+12
19778,آرمان,1397-12-28,بین المللی سرمایه گذاری ایرانیان,38738476064699383,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,66.0,7.82,93931306,private,1.500000e+09,1.116000e+12
19779,آرمان,1397-12-28,سرمایه گذاری آینده نگر شرق,38738476064699383,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,66.0,5.72,68753850,investment,1.500000e+09,1.116000e+12
19780,آرمان,1397-12-28,سرمایه گذاری دلیران پارس,38738476064699383,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,66.0,3.06,36797862,investment,1.500000e+09,1.116000e+12
19781,آرمان,1397-12-28,سرمایه گذاری سهم یاسان بهداد,38738476064699383,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,66.0,4.43,53218443,investment,1.500000e+09,1.116000e+12
19782,آرمان,1397-12-28,وزمین,38738476064699383,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,66.0,19.78,237407350,bank,1.500000e+09,1.116000e+12
19783,آرمان,1397-12-28,وگردش,38738476064699383,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,66.0,15.51,186200000,bank,1.500000e+09,1.116000e+12
19784,آرمان,1397-12-28,پاتین بین الملل پردیس,38738476064699383,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,66.0,12.00,144100000,private,1.500000e+09,1.116000e+12
19785,آرمان,1397-12-28,پیشگامان توسعه شهری ایرانیان,38738476064699383,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,66.0,2.25,27004640,private,1.500000e+09,1.116000e+12
